In [ ]:
# Databricks notebook source

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# Initialize SparkSession if not already available
spark = SparkSession.builder.appName("Orders Transform").getOrCreate()

# Define Unity Catalog catalog and schema names
# IMPORTANT: Replace 'your_catalog' and 'your_schema' with your actual desired Unity Catalog catalog and schema names.
catalog_name = "main_catalog"
schema_name = "silver_orders_schema"

# Create catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

orders_input_table = "main_catalog.bronze_orders_schema.orders_raw"
users_input_table = "main_catalog.silver_users_schema.users_transformed"
products_input_table = "main_catalog.silver_products_schema.products_transformed"

# Define the Unity Catalog output table name
output_table_name = f"{catalog_name}.{schema_name}.orders_transformed"

# Read the raw data from the Unity Catalog tables
df_orders = spark.read.table(orders_input_table)
df_users = spark.read.table(users_input_table)
df_products = spark.read.table(products_input_table)

# Perform transformations
df_transformed = df_orders \
    .withColumn("quantity", col("quantity").cast("int")) \
    .withColumn("order_date", to_timestamp(col("order_date"))) \
    .join(df_users, "user_id", "left") \
    .join(df_products, "product_id", "left") \
    .select("order_id", "user_id", "product_id", "quantity", "order_date", "name", "brand", "price")

# Write the transformed data to a Unity Catalog Delta table
# Ensure you have appropriate permissions to write to the specified catalog and schema.
df_transformed.write.format("delta").mode("overwrite").saveAsTable(output_table_name)

print(f"Successfully transformed orders data and saved to Unity Catalog table: {output_table_name}")
